# MNIST with CNN

Hello, lets apply CNN on mnist dataset using Tensorflow, MNIST dataset consist of approx 60k train and 10k test handwritten images of numbers 0 to 9, it is one of the classic dataset which is already present in tensorflow, so lets get started, very first step is to import tensorflow in our notebook so lets do it, import tensorflow as tf

now, lets read mnist dataset from library by writing following lines of code, we have to import input_data from tensorfkow.examples.tutorials.mnist and while reading dataset  just put one_hot = True to encode the labels of the dataset.

In [9]:
import tensorflow as tf

In [10]:
from tensorflow.examples.tutorials.mnist import input_data

In [3]:
mnist = input_data.read_data_sets("MNIST_data/",one_hot=True) #to encode the labels of the dataset like for digit 9 its[0,0,0,0,0,0,0,0,1]

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use urllib or similar directly.
Successfully downloaded train-images-idx3-ubyte.gz 9912422 bytes.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data/train-images-idx3-ubyte.gz
Successfully downloaded train-labels-idx1-ubyte.gz 28881 bytes.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Successfully downloaded t10k-images-idx3-ubyte.gz 1648877 bytes.
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Successfully downloaded t10k-labels-idx1-ubyte.gz 4542 bytes.
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py fr

Just like NN, training CNNs is essentially the same once we setup out
Network Layers. The flow follows the following steps
1. Random Weight initialization in the Convolution Kernels
2. Forward Propagates an image through our network (Input -> CONV ->
ReLU -> Pooling -> FC
3. Calculate the Total Error e.g. say we got an output of [0.2, 0.4, 0.4] while
the true probabilities were [0, 1, 0]
4. Use Back Propagation to update our gradients (i.e. the weights in the
kernel filters) via gradient descent.
5. Keep propagating all images through our network till an Epoch is
complete
6. Keep completing Epochs till our loss and accuracy are satisfactory 

before starting lets create some helper functions to initialize random weights for fully connected or convolutional layers,we leave the shape attribute as a parameter for this, so lets have the code

### Helper Functions

Function to help intialize random weights for fully connected or convolutional layers, we leave the shape attribute as a parameter for this.

In [4]:
def init_weights(shape):
    init_random_dist = tf.random_normal(shape, stddev=0.1)
    return tf.Variable(init_random_dist)

Same as init_weights, but for the biases

In [5]:
def init_bias(shape):
    init_bias_vals = tf.constant(0.1, shape=shape)
    return tf.Variable(init_bias_vals)

Lets start creating layers, first is conv2D by using inbult tf function and having input parameters as x(input) and W(weights)
what it does,it takes an input tensor and input kernal/filter and perform convolution on it  depending on what strides and padding we provide, stride = 1's in every direction and padding is "same".which means all zeros padding


In [6]:
def conv2d(x, W):
    return tf.nn.conv2d(x, W, strides=[1, 1, 1, 1], padding='SAME')

In [ ]:
# x --> input tensor
#[batch,H,W,channels], 
# w--> [filter height,filter width,channel_in and channel_out]


Create a max pooling layer, again using built in TF functions:

Performs the max pooling on the input.

    Args:
      value: A 4-D `Tensor` with shape `[batch, height, width, channels]` and
        type `tf.float32`.
      ksize: A list of ints that has length >= 4.  The size of the window for
        each dimension of the input tensor.
      strides: A list of ints that has length >= 4.  The stride of the sliding
        window for each dimension of the input tensor.
      padding: A string, either `'VALID'` or `'SAME'`. 

In [7]:
def max_pool_2by2(x):
    return tf.nn.max_pool(x, ksize=[1, 2, 2, 1],
                          strides=[1, 2, 2, 1], padding='SAME')

Using the conv2d function, we'll return an actual convolutional layer here that uses an ReLu activation.

In [8]:
def convolutional_layer(input_x, shape):
    W = init_weights(shape)
    b = init_bias([shape[3]])
    return tf.nn.relu(conv2d(input_x, W) + b)

This is a normal fully connected layer

In [9]:
def normal_full_layer(input_layer, size):
    input_size = int(input_layer.get_shape()[1])
    W = init_weights([input_size, size])
    b = init_bias([size])
    return tf.matmul(input_layer, W) + b

### Placeholders

lets create some placeholders for input and labels, None to fill batchsize at session run time and 784 coz image is 24*24
y_true are labels which are one hot encoded

In [10]:
x = tf.placeholder(tf.float32,shape=[None,784])

In [11]:
y_true = tf.placeholder(tf.float32,shape=[None,10])

After that lets create layers image_layer which will be our input, lets take it as x_image and reshaping is to convert it into image again as  we flattened an images into array in above step

### Layers

In [12]:
x_image = tf.reshape(x,[-1,28,28,1])

In [13]:
# Using a 6by6 filter here, you can play around with the filter size
# You can change the 32 output, that essentially represents the amount of filters used
# You need to pass in 32 to the next input though, the 1 comes from the original input of 
# a single image.
convo_1 = convolutional_layer(x_image,shape=[6,6,1,32])
convo_1_pooling = max_pool_2by2(convo_1)

In [14]:
# Using a 6by6 filter here you can play around with the filter size
# You can actually change the 64 output if you want, you can think of that as a representation
# of the amount of 6by6 filters used.
convo_2 = convolutional_layer(convo_1_pooling,shape=[6,6,32,64])
convo_2_pooling = max_pool_2by2(convo_2)

In [15]:
# Why 7 by 7 image? Because we did 2 pooling layers, so (28/2)/2 = 7
# 64 then just comes from the output of the previous Convolution
convo_2_flat = tf.reshape(convo_2_pooling,[-1,7*7*64])
full_layer_one = tf.nn.relu(normal_full_layer(convo_2_flat,1024))# we want 1024 neurons

In [16]:
# NOTE THE PLACEHOLDER HERE!
hold_prob = tf.placeholder(tf.float32)
full_one_dropout = tf.nn.dropout(full_layer_one,keep_prob=hold_prob)

In [17]:
y_pred = normal_full_layer(full_one_dropout,10)

### Loss Function

In [18]:
cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y_true,logits=y_pred))

Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See @{tf.nn.softmax_cross_entropy_with_logits_v2}.



### Optimizer

In [19]:
optimizer = tf.train.AdamOptimizer(learning_rate=0.0001)
train = optimizer.minimize(cross_entropy)

### Intialize Variables

In [20]:
init = tf.global_variables_initializer()

### Session

In [21]:
steps = 5000

with tf.Session() as sess:
    
    sess.run(init)
    
    for i in range(steps):
        
        batch_x , batch_y = mnist.train.next_batch(50)
        
        sess.run(train,feed_dict={x:batch_x,y_true:batch_y,hold_prob:0.5})
        
        # PRINT OUT A MESSAGE EVERY 100 STEPS
        if i%100 == 0:
            
            print('Currently on step {}'.format(i))
            print('Accuracy is:')
            # Test the Train Model
            matches = tf.equal(tf.argmax(y_pred,1),tf.argmax(y_true,1))

            acc = tf.reduce_mean(tf.cast(matches,tf.float32)) # to cast list of booleans into floats to find out the accuracy

            print(sess.run(acc,feed_dict={x:mnist.test.images,y_true:mnist.test.labels,hold_prob:1.0})) # no neurons dropped at the time of testing
            print('\n')

Currently on step 0
Accuracy is:
0.0858


Currently on step 100
Accuracy is:
0.8424


Currently on step 200
Accuracy is:
0.9119


Currently on step 300
Accuracy is:
0.9288


Currently on step 400
Accuracy is:
0.9407


Currently on step 500
Accuracy is:
0.9474


Currently on step 600
Accuracy is:
0.9509


Currently on step 700
Accuracy is:
0.9525


Currently on step 800
Accuracy is:
0.9585


Currently on step 900
Accuracy is:
0.9606


Currently on step 1000
Accuracy is:
0.9611


Currently on step 1100
Accuracy is:
0.9644


Currently on step 1200
Accuracy is:
0.966


Currently on step 1300
Accuracy is:
0.9663


Currently on step 1400
Accuracy is:
0.9689


Currently on step 1500
Accuracy is:
0.9701


Currently on step 1600
Accuracy is:
0.973


Currently on step 1700
Accuracy is:
0.9745


Currently on step 1800
Accuracy is:
0.9697


Currently on step 1900
Accuracy is:
0.9773


Currently on step 2000
Accuracy is:
0.9749


Currently on step 2100
Accuracy is:
0.9773


Currently on step 2200
A

Accuracy is inc as no of epochs increases which means our model is working fine and at the end we have accuracy of 98%, which is preety good.
Thanks

In [ ]:
#predicted [3,4] = actual[3,9]
#[true,false]
#[1.0,0.0]
#0.5  accu is 50%